In [1]:
# pip install transformers

# https://github.com/00ddya3/20232_DL/blob/74eba1147edae18e64b1b5367e05f58a2e002157/1030/BERT_Kor_movie_reviews_sentiment_fine_tuning_KcBERT.ipynb#L51
# 더 간단한 코드

In [2]:
import transformers
transformers.__version__

'4.35.2'

In [3]:
# pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [4]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from sklearn import preprocessing
from transformers import AutoTokenizer, TFGPT2Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import os
import sys

print(os.getcwd())

/content


In [7]:
import pandas as pd
train_data = pd.read_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/train.hate.csv')
train_data_cl1 = pd.read_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/train.hate.csv')
train_data_cl2 = pd.read_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/train.hate.csv')

# train_newstitle = pd.read_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/train.news_title.txt', sep='\t', names=['news_title'])

unlabeled_comments = pd.read_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/unlabeled_comments.txt', sep='\t', names=['unlabeled_comments'])

dev_data = pd.read_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/dev.hate.csv')
test_data = pd.read_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/test.hate.no_label.csv')

# , sep='\t', quoting=3

In [8]:
# unlabeled_comments

In [9]:
train_data.head()

,comments,label
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,hate
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,none
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",hate
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",none
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,hate


In [10]:
LABEL_DIC = {
    'none': 0,
    'offensive': 1,
    'hate': 2,
}

train_data['label'] = train_data['label'].map(lambda x: LABEL_DIC[x])
train_data_cl1['label'] = train_data_cl1['label'].map(lambda x: LABEL_DIC[x])
train_data_cl2['label'] = train_data_cl2['label'].map(lambda x: LABEL_DIC[x])

dev_data['label'] = dev_data['label'].map(lambda x: LABEL_DIC[x])

In [11]:
# 데이터 정제

import string

## remove basic punctuation
def remove_punc(text):
    text_nopunc = "".join([char for char in text if char not in string.punctuation])
    return text_nopunc

train_data_cl1['comments'] = train_data_cl1['comments'].apply(lambda x: remove_punc(x))

# train_data['news_title'] = train_data['news_title'].apply(lambda x: remove_punc(x))

# dev_data['comments'] = dev_data['comments'].apply(lambda x: remove_punc(x))

# test_data['comments'] = test_data['comments'].apply(lambda x: remove_punc(x))

In [12]:
train_data_cl1['comments']

0       현재 호텔주인 심정 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네
1         한국적인 미인의 대표적인 분너무나 곱고아름다운모습그모습뒤의 슬픔을 미처 알지못했네요ㅠ
2       못된 넘들남의 고통을 즐겼던 넘들이젠 마땅한 처벌을 받아야지그래야 공정한 사회지심은...
3                        12화 어설펐는데 34화 지나서부터는 갈수록 너무 재밌던데
4         1 사람 얼굴 손톱으로 긁은것은 인격살해이고2 동영상이 몰카냐 메걸리안들 생각이 없노
                              ...                        
7891                                           힘내세요 응원합니다
7892                                   힘내세요삼가 고인의 명복을 빕니다
7893                                     힘내세용  항상 응원합니닷  
7894        힘내소연기로 답해요나도 53살 인데 이런일 저런일 다 있더라구요인격을 믿습니다홨팅
7895                                   힘들면 관뒀어야지 그게 현명한거다
Name: comments, Length: 7896, dtype: object

In [13]:
import re

# remove all punctuations except korean, english, and number
def cleanse(text):
    pattern = re.compile(r'\s+')
    text = re.sub(pattern, ' ', text)
    text = re.sub('[a-zA-Z]', '', text)
    text = re.sub('[ㄱ-ㅎㅏ-ㅣ]+', '', text)
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', text)
    return text

train_data_cl2['comments'] = train_data_cl2['comments'].apply(cleanse)
# train['news_title'] = train['news_title'].apply(cleanse)

# dev_data['comments'] = dev_data['comments'].apply(cleanse)

# test_data['comments'] = test_data['comments'].apply(cleanse)

train_data.head()

,comments,label
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,2
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,0
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",2
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",0
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,2


In [14]:
# train_data_list = []

# for p, q, r in zip(train_data['comments'], train_data['clean_comments'], train_data['clean2_comments'])  :
#     data = []
#     data.append(q + q + r)
#     # data.append(str(label))

#     train_data_list.append(data)

# dev_data_list = []

# for q, label in zip(dev_data['comments'], dev_data['label'])  :
#     data = []
#     data.append(q)
#     data.append(str(label))

#     dev_data_list.append(data)

In [15]:
# tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>')

# tokenizer = BertTokenizer.from_pretrained("beomi/kcbert-base")
tokenizer = AutoTokenizer.from_pretrained('beomi/kcbert-base', bos_token='</s>', eos_token='</s>', pad_token='<pad>', mask_token='<mask>')

In [16]:
# print(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))
# print(tokenizer.tokenize("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))
# tokenizer.decode(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

# for elem in tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"):
#   print(tokenizer.decode(elem))

# print(tokenizer.tokenize("전율을 일으키는 영화. 다시 보고싶은 영화"))

# print(tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화"))

# for elem in tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화"):
#   print(tokenizer.decode(elem))

# for elem in tokenizer.encode("happy birthday~!"):
#   print(tokenizer.decode(elem))

# print(tokenizer.decode(0))
# print(tokenizer.decode(1))
# print(tokenizer.decode(2))
# print(tokenizer.decode(3))
# print(tokenizer.decode(4))

In [17]:
max_seq_len = 140

In [18]:
# encoded_result = tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화", max_length=max_seq_len, padding='max_length')
# print(encoded_result)
# print('길이 :', len(encoded_result))

# print(tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화", max_length=140))

In [19]:
def convert_examples_to_features(comments, max_seq_len, tokenizer):

    input_ids = []

    for sent1 in tqdm(comments, total = len(comments)):
        # bos_token = [tokenizer.bos_token]
        # eos_token = [tokenizer.eos_token]
        # sent1_tokens = bos_token + tokenizer.tokenize(sent1) + eos_token + ['']

        sent1_tokens = [tokenizer.bos_token] + tokenizer.tokenize(sent1) + [tokenizer.eos_token]

        tokens = sent1_tokens
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        input_ids.append(input_id)

    input_ids = np.array(input_ids, dtype=int)

    return input_ids

In [20]:
new_train_data = pd.concat([train_data, train_data_cl1, train_data_cl2])

new_train_data['comments'][0]

new_train_data.shape

(23688, 2)

In [21]:
X_train = convert_examples_to_features(new_train_data['comments'], max_seq_len = max_seq_len, tokenizer = tokenizer)

100%|██████████| 23688/23688 [00:07<00:00, 3139.67it/s]


In [22]:
# new_train_data
# index 중복 문제 있을지도

In [23]:
# 최대 길이: 128
input_id = X_train[0]

print('단어에 대한 정수 인코딩 :', input_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))

단어에 대한 정수 인코딩 : [30000    11  8979 16099 17784 20383    12  2170  9168   591  1293  4384
 26536  4113   595 29540 12020 16099  4227  8007 13999  7972  9229  8249
 15042  4282  4011    17    17    17    17 30000 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001]
각 인코딩의 길이 : 140
정수 인코딩 복원 : </s> ( 현재 호텔주인 심정 ) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네.... </s> <pad> <pad> <pad> <pad> <pad> <pad>

In [24]:
X_dev = convert_examples_to_features(dev_data['comments'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 471/471 [00:00<00:00, 3198.79it/s]


In [25]:
# 최대 길이: 128
input_id = X_dev[0]

print('단어에 대한 정수 인코딩 :',input_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))

단어에 대한 정수 인코딩 : [30000 26295  4184  9349  4496  4057  9246  9918    17  2881  4375 23645
  7968 12307  4020    17 30000 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001]
각 인코딩의 길이 : 140
정수 인코딩 복원 : </s> 송중기 시대극은 믿고본다. 첫회 신선하고 좋았다. </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

In [26]:
X_test = convert_examples_to_features(test_data['comments'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 974/974 [00:00<00:00, 3244.79it/s]


In [27]:
train_label = new_train_data['label'].tolist()
dev_label = dev_data['label'].tolist()
# test_label = test_data['label'].tolist()

# train_label

In [28]:
idx_encode = preprocessing.LabelEncoder()
idx_encode.fit(train_label)

y_train = idx_encode.transform(train_label) # 주어진 고유한 정수로 변환
y_dev = idx_encode.transform(dev_label) # 고유한 정수로 변환
# y_test = idx_encode.transform(test_label) # 고유한 정수로 변환

label_idx = dict(zip(list(idx_encode.classes_), idx_encode.transform(list(idx_encode.classes_))))
idx_label = {value: key for key, value in label_idx.items()}
print(label_idx)
print(idx_label)

{0: 0, 1: 1, 2: 2}
{0: 0, 1: 1, 2: 2}


In [29]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification

model = TFBertForSequenceClassification.from_pretrained("beomi/kcbert-base", num_labels=3, from_pt=True)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
# model = TFGPT2Model.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

In [31]:
max_seq_len = 140

In [32]:
# input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
# outputs = model([input_ids_layer])

# print(outputs)
# print(outputs[0])
# print(outputs[1])
# print(outputs[0][:, -1])

In [33]:
class TFGPT2ForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super(TFGPT2ForSequenceClassification, self).__init__()
        self.gpt = TFGPT2Model.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(num_labels,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='softmax',
                                                name='classifier')

    def call(self, inputs):
        outputs = self.gpt(input_ids=inputs)
        cls_token = outputs[0][:, -1]
        prediction = self.classifier(cls_token)

        return prediction

TPU 사용법 : https://wikidocs.net/119990

In [34]:
# TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [35]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [36]:
with strategy.scope():
  model = TFBertForSequenceClassification.from_pretrained("beomi/kcbert-base", num_labels=3, from_pt=True, hidden_dropout_prob = 0.6)

  # hidden_dropout_prob = 0.3

  # model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2", num_labels=3)

  optimizer = tf.keras.optimizers.Adam(learning_rate=1e-6)
  loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
  model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])


# https://huggingface.co/transformers/v2.10.0/model_doc/bert.html#tfbertforsequenceclassification

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
from tensorflow.keras.utils import to_categorical

y_train_encoded = to_categorical(y_train)
y_dev_encoded = to_categorical(y_dev)

In [38]:
early_stopping = EarlyStopping(
    monitor="val_accuracy",
    min_delta = 0.001,
    patience = 2)

history = model.fit(X_train, y_train_encoded, epochs = 30, batch_size = 64, validation_data=(X_dev, y_dev_encoded))
#, callbacks = [early_stopping])

# max = 0.6858 / 0.7070
# 6391 (2e-6)
# 0.6539
#

Epoch 1/30
371/371 [==============================] - 107s 131ms/step - loss: 0.6411 - accuracy: 0.4081 - val_loss: 0.6587 - val_accuracy: 0.3397
Epoch 2/30
371/371 [==============================] - 43s 117ms/step - loss: 0.6301 - accuracy: 0.4262 - val_loss: 0.6577 - val_accuracy: 0.3397
Epoch 3/30
371/371 [==============================] - 43s 116ms/step - loss: 0.6290 - accuracy: 0.4317 - val_loss: 0.6529 - val_accuracy: 0.3503
Epoch 4/30
371/371 [==============================] - 43s 116ms/step - loss: 0.6271 - accuracy: 0.4345 - val_loss: 0.6545 - val_accuracy: 0.3652
Epoch 5/30
371/371 [==============================] - 43s 117ms/step - loss: 0.6238 - accuracy: 0.4375 - val_loss: 0.6409 - val_accuracy: 0.4034
Epoch 6/30
371/371 [==============================] - 43s 115ms/step - loss: 0.6177 - accuracy: 0.4494 - val_loss: 0.6039 - val_accuracy: 0.4650
Epoch 7/30
371/371 [==============================] - 43s 117ms/step - loss: 0.5925 - accuracy: 0.4981 - val_loss: 0.5372 - val_a

In [39]:
# 토큰화 된 상태
# X_test

In [40]:
output = model.predict(X_test)

31/31 [==============================] - 8s 57ms/step


In [41]:
output.logits

array([[ 1.4471495 , -1.9462193 , -3.2452853 ],
       [ 3.4587626 , -3.4277241 , -3.7995949 ],
       [ 0.88528955, -1.2147934 , -3.2302592 ],
       ...,
       [ 0.8102726 , -1.1247162 , -2.63514   ],
       [-3.9619987 , -1.1871697 ,  1.3733941 ],
       [ 3.2506905 , -3.352115  , -4.085548  ]], dtype=float32)

In [42]:
import torch

outputTensor = torch.Tensor(output.logits)

In [43]:
# outputTensor = nn.Softmax(dim=1)(outputTensor)
pred = torch.argmax(outputTensor, dim=-1)

In [44]:
pred

tensor([0, 0, 0, 0, 0, 0, 1, 1, 0, 2, 2, 0, 0, 0, 2, 1, 0, 0, 2, 0, 1, 2, 2, 1,
        0, 0, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 2, 0, 0, 2, 0, 0,
        0, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 1, 1, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 2, 0, 1, 2, 0, 2, 0, 0, 2,
        2, 0, 1, 0, 0, 0, 2, 1, 0, 0, 2, 0, 1, 0, 0, 0, 2, 0, 0, 2, 1, 0, 0, 0,
        0, 1, 2, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        0, 2, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 2, 1, 0, 1, 2, 1, 2, 2, 1, 0, 0,
        0, 1, 2, 1, 0, 0, 1, 0, 0, 0, 2, 1, 2, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 2, 0, 1, 2, 0, 2, 0, 0, 0, 0, 0, 0,
        2, 0, 0, 1, 1, 0, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 2, 1, 2,
        2, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 2, 1, 0, 0, 1, 0, 2,
        0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0,
        0, 1, 0, 1, 0, 0, 1, 0, 0, 2, 2,

In [45]:
test_label = [int(k.cpu()) for k in pred]
test_data['label'] = test_label

In [46]:
test_data

,comments,label
0,ㅋㅋㅋㅋ 그래도 조아해주는 팬들 많아서 좋겠다 ㅠㅠ 니들은 온유가 안만져줌 ㅠㅠ,0
1,둘다 넘 좋다~행복하세요,0
2,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,0
3,원곡생각하나도 안나고 러블리즈 신곡나온줄!!! 너무 예쁘게 잘봤어요,0
4,장현승 얘도 참 이젠 짠하다...,0
...,...,...
969,대박 게스트... 꼭 봐야징~ 컨셉이 바뀌니깐 재미지넹,0
970,성형으로 다 뜯어고쳐놓고 예쁜척. 성형 전 니 얼굴 다 알고있다. 순자처럼 된장냄새...,2
971,분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...,0
972,입에 손가릭이 10개 있으니 징그럽다,2


In [47]:
test_data.to_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/submission/KcBERT_miracle.csv', index=None)